In [1]:
# Install necessary libraries
!pip install open3d numpy tensorflow matplotlib pyntcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.3/346.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7

In [2]:
import numpy as np
import open3d as o3d
from pyntcloud import PyntCloud
import pandas as pd

In [3]:
# Generate a tree (trunk and leaves)
def generate_tree(center, height, trunk_radius, leaves_radius, trunk_points=1000, leaves_points=2000):
    """Generate realistic tree point cloud."""
    # Trunk (cylinder)
    theta = np.random.uniform(0, 2 * np.pi, trunk_points)
    z = np.random.uniform(0, height, trunk_points)
    x = trunk_radius * np.cos(theta) + center[0]
    y = trunk_radius * np.sin(theta) + center[1]
    trunk = np.vstack([x, y, z]).T

    # Leaves (sphere at the top)
    phi = np.random.uniform(0, np.pi, leaves_points)
    theta = np.random.uniform(0, 2 * np.pi, leaves_points)
    r = np.random.uniform(0, leaves_radius, leaves_points)
    x = r * np.sin(phi) * np.cos(theta) + center[0]
    y = r * np.sin(phi) * np.sin(theta) + center[1]
    z = r * np.cos(phi) + center[2] + height
    leaves = np.vstack([x, y, z]).T

    return trunk, leaves

In [4]:
# Generate a building (cube)
def generate_building(center, width, depth, height, points=5000):
    """Generate realistic building point cloud."""
    x = np.random.uniform(center[0] - width / 2, center[0] + width / 2, points)
    y = np.random.uniform(center[1] - depth / 2, center[1] + depth / 2, points)
    z = np.random.uniform(center[2], center[2] + height, points)
    building = np.vstack([x, y, z]).T
    return building

In [5]:
# Generate a car (rectangular cuboid with smaller height)
def generate_car(center, length, width, height, points=2000):
    """Generate realistic car point cloud."""
    x = np.random.uniform(center[0] - length / 2, center[0] + length / 2, points)
    y = np.random.uniform(center[1] - width / 2, center[1] + width / 2, points)
    z = np.random.uniform(center[2], center[2] + height, points)
    car = np.vstack([x, y, z]).T
    return car

In [6]:
# Generate a person (cylinder)
def generate_person(center, height, radius, points=1000):
    """Generate realistic person point cloud."""
    theta = np.random.uniform(0, 2 * np.pi, points)
    z = np.random.uniform(0, height, points)
    x = radius * np.cos(theta) + center[0]
    y = radius * np.sin(theta) + center[1]
    person = np.vstack([x, y, z]).T
    return person

In [12]:
def generate_fence(center, length, height, num_bars, points_per_bar=200):
    x_positions = np.linspace(center[0] - length / 2, center[0] + length / 2, num_bars)
    z_positions = np.linspace(center[2], center[2] + height, points_per_bar)

    # Generate points for each bar
    bars = []
    for x in x_positions:
        x_coords = np.full(points_per_bar, x)
        y_coords = np.full(points_per_bar, center[1])
        z_coords = z_positions
        bar = np.vstack([x_coords, y_coords, z_coords]).T
        bars.append(bar)

    fence = np.vstack(bars)
    return fence

In [13]:
# Generate a lamppost (cylinder)
def generate_lamppost(center, height, radius, points=1500):
    """Generate a lamppost point cloud."""
    theta = np.random.uniform(0, 2 * np.pi, points)
    z = np.random.uniform(0, height, points)
    x = radius * np.cos(theta) + center[0]
    y = radius * np.sin(theta) + center[1]
    lamppost = np.vstack([x, y, z]).T
    return lamppost

In [14]:
# Generate a bench (rectangular cuboid with smaller dimensions)
def generate_bench(center, length, width, height, points=1000):
    """Generate a bench point cloud."""
    x = np.random.uniform(center[0] - length / 2, center[0] + length / 2, points)
    y = np.random.uniform(center[1] - width / 2, center[1] + width / 2, points)
    z = np.random.uniform(center[2], center[2] + height, points)
    bench = np.vstack([x, y, z]).T
    return bench


In [15]:
# Combine all objects into a single point cloud
def combine_objects_with_labels(objects):
    """Combine multiple objects into one point cloud with labels."""
    all_points = []
    all_colors = []
    all_labels = []
    label_counter = 1  # Start labels from 1 for each unique object type

    for obj_points, obj_color in objects:
        all_points.append(obj_points)
        colors = np.tile(obj_color, (len(obj_points), 1))
        all_colors.append(colors)
        labels = np.full(len(obj_points), label_counter, dtype=np.int32)
        all_labels.append(labels)
        label_counter += 1

    # Concatenate points, colors, and labels
    combined_points = np.vstack(all_points)
    combined_colors = np.vstack(all_colors)
    combined_labels = np.hstack(all_labels)

    return combined_points, combined_colors, combined_labels

In [16]:
# Generate individual objects
tree_trunk, tree_leaves = generate_tree(center=[0, 5, 0], height=3, trunk_radius=0.3, leaves_radius=1.5)
building = generate_building(center=[-10, -5, 0], width=5, depth=5, height=10)
car = generate_car(center=[5, 0, 0], length=4, width=2, height=1.5)
person = generate_person(center=[3, 3, 0], height=1.8, radius=0.3)
fence = generate_fence(center=[8, -3, 0], length=6, height=1.2, num_bars=10)
lamppost = generate_lamppost(center=[-5, 7, 0], height=5, radius=0.2)
bench = generate_bench(center=[6, -6, 0], length=3, width=1, height=0.8)

In [17]:
# Assign colors and labels to objects
objects = [
    (tree_trunk, [0.5, 0.25, 0.1]),  # Brown trunk
    (tree_leaves, [0.1, 0.8, 0.1]),  # Green leaves
    (building, [0.6, 0.6, 0.6]),     # Gray building
    (car, [0.9, 0.1, 0.1]),          # Red car
    (person, [0.1, 0.1, 0.8]),       # Blue person
    (fence, [0.7, 0.7, 0.7]),        # Light gray fence
    (lamppost, [0.4, 0.4, 0.4]),     # Dark gray lamppost
    (bench, [0.8, 0.5, 0.2])         # Brown bench
]

# Combine objects into a single point cloud
points, colors, labels = combine_objects_with_labels(objects)

In [18]:
# Split into training and validation datasets
train_ratio = 0.8
split_idx = int(len(points) * train_ratio)

train_points, val_points = points[:split_idx], points[split_idx:]
train_colors, val_colors = colors[:split_idx], colors[split_idx:]
train_labels, val_labels = labels[:split_idx], labels[split_idx:]

In [19]:
# Save training data
train_cloud = o3d.geometry.PointCloud()
train_cloud.points = o3d.utility.Vector3dVector(train_points)
train_cloud.colors = o3d.utility.Vector3dVector(train_colors)
o3d.io.write_point_cloud("training_point_cloud.ply", train_cloud)
np.savetxt("training_labels.txt", train_labels, fmt='%d', header="Labels for training points")

In [20]:
# Save validation data
val_cloud = o3d.geometry.PointCloud()
val_cloud.points = o3d.utility.Vector3dVector(val_points)
val_cloud.colors = o3d.utility.Vector3dVector(val_colors)
o3d.io.write_point_cloud("validation_point_cloud.ply", val_cloud)
np.savetxt("validation_labels.txt", val_labels, fmt='%d', header="Labels for validation points")

In [21]:
# Output details
print("Training and validation datasets generated and saved.")
print("Training point cloud saved as 'training_point_cloud.ply'")
print("Validation point cloud saved as 'validation_point_cloud.ply'")
print("Training labels saved as 'training_labels.txt'")
print("Validation labels saved as 'validation_labels.txt'")

Training and validation datasets generated and saved.
Training point cloud saved as 'training_point_cloud.ply'
Validation point cloud saved as 'validation_point_cloud.ply'
Training labels saved as 'training_labels.txt'
Validation labels saved as 'validation_labels.txt'
